# Rat OGD Image Preprocessing

Teng-Jui Lin

Created: 2022-05-20

Edited: 2022-08-18

Objective: Preprocess the given unlabeled images in `.npy` to labeled images in `.tif` for further VAMPIRE analysis.

Assumption: The folder only contains `.npy` files of interest for image preprocessing.

## Read, label, and write images

In [1]:
!where python
!python --version

/opt/miniconda3/envs/vampire-analysis/bin/python
Python 3.9.12


In [2]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage
import vampire as vp
from skimage import io

In [3]:
# set up filepaths to .npy files
# assume the folder has only .npy files of interest
data_npy_dir = r"/Users/tlin/Files/2019-2023 University of Washington/_nance-lab/research-data/microfiber/rat-ogd/1-thresholded-data-npy"
filenames = np.array(os.listdir(data_npy_dir))
os_path_join = np.vectorize(os.path.join)
filepaths = os_path_join(data_npy_dir, filenames)

# filter for .npy files - avoid hidden file problem like .DS_Store
# separate otsu and yen thereholded images
npy_mask = pd.Series(filepaths).str.contains("\.npy").to_numpy()
otsu_mask = pd.Series(filepaths).str.contains("otsu").to_numpy()
yen_mask = pd.Series(filepaths).str.contains("yen").to_numpy()
otsu_filepaths = filepaths[npy_mask & otsu_mask]
yen_filepaths = filepaths[npy_mask & yen_mask]

In [4]:
def load_imgs(filepaths):
    """Filepaths are paths to images. Returns image ndarrays."""
    imgs = []
    for filepath in filepaths:
        imgs.append(skimage.measure.label(np.load(filepath)))
    return imgs

In [5]:
otsu_imgs = load_imgs(otsu_filepaths)
yen_imgs = load_imgs(yen_filepaths)

In [6]:
# output paths of .tif files
otsu_img_paths = (
    pd.Series(otsu_filepaths)
    .str.replace("1-thresholded-data-npy", "2-thresholded-data-tif")
    .str.replace("npy", "tif")
    .to_numpy()
)
yen_img_paths = (
    pd.Series(yen_filepaths)
    .str.replace("1-thresholded-data-npy", "2-thresholded-data-tif")
    .str.replace("npy", "tif")
    .to_numpy()
)

In [7]:
def save_imgs(img_set, save_img_paths):
    for i in range(len(img_set)):
        io.imsave(save_img_paths[i], img_set[i])

In [8]:
import warnings

# expect low contrast .tif images
warnings.filterwarnings("ignore", category=UserWarning)
save_imgs(otsu_imgs, otsu_img_paths)
save_imgs(yen_imgs, yen_img_paths)